In [1]:
# Install dependencies
!pip install pandas numpy requests beautifulsoup4 selenium scikit-learn statsmodels tensorflow torch transformers coinmarketcap-api alpha_vantage yfinance


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 28.0 MB/s eta 0:00:00
  Cre

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
import pandas as pd

# Fetch data from CoinGecko
def get_coingecko_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,  # Top 10 cryptocurrencies
        "page": 1,
        "sparkline": False
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)[["id", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
        return df
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Get the data
crypto_df = get_coingecko_data()
crypto_df.head()


,id,symbol,current_price,market_cap,total_volume,price_change_percentage_24h
0,bitcoin,btc,88783.000000,1759775902217,91372386107,-4.08856
1,ethereum,eth,2509.530000,302271638022,44334969035,-0.67616
2,tether,usdt,0.999011,142205318164,70373785588,-0.08811
3,ripple,xrp,2.330000,134956635882,11276858702,1.85462
4,binancecoin,bnb,626.070000,91242500052,1991147105,2.11336


In [4]:
def get_cryptocompare_data(symbol="BTC", currency="USD", limit=50):
    url = f"https://min-api.cryptocompare.com/data/v2/histoday"
    params = {
        "fsym": symbol,
        "tsym": currency,
        "limit": limit,  # Fetch last 50 days of data
        "api_key": "CRYPTOCOMPARE_API_KEY"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()["Data"]["Data"]
        df = pd.DataFrame(data)
        df["time"] = pd.to_datetime(df["time"], unit="s")  # Convert timestamp
        return df
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Fetch Bitcoin historical data
btc_df = get_cryptocompare_data("BTC")
btc_df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2025-01-06,102530.34,97908.09,98346.97,46146.96,4.654078e+09,102282.20,direct,
1,2025-01-07,102747.54,96112.95,102282.20,54319.14,5.348080e+09,96942.47,direct,
2,2025-01-08,97251.53,92488.45,96942.47,57721.92,5.480363e+09,95051.06,direct,
3,2025-01-09,95345.44,91197.56,95051.06,50519.28,4.696698e+09,92548.12,direct,
4,2025-01-10,95845.36,92206.53,92548.12,55479.83,5.217668e+09,94710.29,direct,


In [5]:
pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=d9e079a92154809bdeec2b3a982e75f993a2097170d5da5f0b14cab5c790418c
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [6]:
import feedparser

url = "https://cointelegraph.com/rss"
feed = feedparser.parse(url)

for entry in feed.entries[:5]:  # Get the latest 5 news articles
    print("Title:", entry.title)
    print("Published:", entry.published)
    print("Link:", entry.link)
    print()


Title: 3 reasons why Solana (SOL) price is 50% down from its all-time high
Published: Tue, 25 Feb 2025 22:45:00 +0000
Link: https://cointelegraph.com/news/3-reasons-why-solana-sol-price-is-down-50-from-its-all-time-high?utm_source=rss_feed&utm_medium=rss&utm_campaign=rss_partner_inbound

Title: Solana price unlikely to bounce back quickly: Here’s why
Published: Tue, 25 Feb 2025 21:46:08 +0000
Link: https://cointelegraph.com/news/solana-price-unlikely-to-bounce-back-quickly-here-s-why?utm_source=rss_feed&utm_medium=rss&utm_campaign=rss_partner_inbound

Title: Here’s what happened in crypto today
Published: Tue, 25 Feb 2025 21:42:32 +0000
Link: https://cointelegraph.com/news/what-happened-in-crypto-today?utm_source=rss_feed&utm_medium=rss&utm_campaign=rss_partner_inbound

Title: US Senator introduces bill to stop crypto ATM fraud
Published: Tue, 25 Feb 2025 21:02:24 +0000
Link: https://cointelegraph.com/news/senator-dick-durbin-legislation-stop-crypto-atm-fraud?utm_source=rss_feed&utm_me

In [13]:
import requests
import pandas as pd

# Fetch data from CoinGecko and save to CSV
def get_coingecko_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,  # Top 10 cryptocurrencies
        "page": 1,
        "sparkline": False
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)[["id", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
        df["timestamp"] = pd.Timestamp.now()  # Add timestamp for tracking

        # Save the data to a CSV file (append if exists)
        df.to_csv("/content/drive/MyDrive/Crypto data/crypto_data.csv", mode="a", header=not pd.io.common.file_exists("/content/drive/MyDrive/Crypto data/crypto_data.csv"), index=False)
        print("Data saved to CSV successfully!")
        return df
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Run function once to test
crypto_df = get_coingecko_data()
crypto_df.head()


Data saved to CSV successfully!


,id,symbol,current_price,market_cap,total_volume,price_change_percentage_24h,timestamp
0,bitcoin,btc,89001.000000,1765012928242,83842850960,-3.94912,2025-02-25 23:14:37.007779
1,ethereum,eth,2517.980000,303493478283,43995119773,-0.85420,2025-02-25 23:14:37.007779
2,tether,usdt,0.998993,142207306655,130245158853,-0.08451,2025-02-25 23:14:37.007779
3,ripple,xrp,2.340000,135509244038,11275960981,1.42010,2025-02-25 23:14:37.007779
4,binancecoin,bnb,626.970000,91532485821,1992757850,1.62932,2025-02-25 23:14:37.007779
